In [1]:

import tensorflow as tf
print(tf.__version__)



2.15.0


In [2]:
pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade tensorflow


Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf
print(tf.__version__)


2.15.0


In [5]:
pip install keras


Note: you may need to restart the kernel to use updated packages.


In [6]:
from keras.layers import Input


In [7]:
from tensorflow.keras.layers import Input
import tensorflow as tf
import pathlib
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, LeakyReLU, Activation, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from numpy import zeros, ones
from numpy.random import randint
from matplotlib import pyplot as plt


In [8]:
import numpy as np
import os
from skimage.transform import resize
from matplotlib import pyplot as plt

# Define file paths
input_dir = r"C:\Users\net pc\Desktop\Thesis Paper 3\Brain tumor dataset\Training\glioma_tumor_inputs"
label_dir = r"C:\Users\net pc\Desktop\Thesis Paper 3\Brain tumor dataset\Training\glioma_tumor_targets"


def load_data(input_dir, label_dir, target_shape):
    input_files = os.listdir(input_dir)
    label_files = os.listdir(label_dir)
    
    # Sort files to ensure corresponding inputs and labels match
    input_files.sort()
    label_files.sort()
    
    trainA = []
    trainB = []
    for i in range(len(input_files)):
        input_img = plt.imread(os.path.join(input_dir, input_files[i]))
        label_img = plt.imread(os.path.join(label_dir, label_files[i]))
        
        # Resize images to target shape if they have different shapes
        if input_img.shape != target_shape:
            input_img = resize(input_img, target_shape)
        if label_img.shape != target_shape:
            label_img = resize(label_img, target_shape)
        
        trainA.append(input_img)
        trainB.append(label_img)
    
    return np.array(trainA), np.array(trainB)



# Define target shape for images (e.g., (256, 256, 3))
target_shape = (256, 256, 3)

# Load data
data = load_data(input_dir, label_dir, target_shape)
print("Data loaded successfully!")


Data loaded successfully!


In [9]:


# Define discriminator
def define_discriminator(image_shape):
    init = RandomNormal(stddev=0.02)
    in_src_image = Input(shape=image_shape, name='input_image')
    in_target_image = Input(shape=image_shape, name='target_image')
    merged = Concatenate()([in_src_image, in_target_image])
    
    # Discriminator architecture
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(patch_out)
    
    model = Model([in_src_image, in_target_image], patch_out)
    model.compile(loss='binary_crossentropy', optimizer='adam', loss_weights=[0.5])
    return model

In [10]:
def define_generator(image_shape=(256,256,3)):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=image_shape)
    
    # Encoder
    e1 = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
    e2 = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e1))
    e2 = BatchNormalization()(e2)
    e3 = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e2))
    e3 = BatchNormalization()(e3)
    e4 = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e3))
    e4 = BatchNormalization()(e4)
    e5 = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e4))
    e5 = BatchNormalization()(e5)
    e6 = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e5))
    e6 = BatchNormalization()(e6)
    e7 = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e6))
    e7 = BatchNormalization()(e7)
    
    # Bottleneck
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e7))
    b = Activation('relu')(b)
    
    # Decoder
    d1 = Conv2DTranspose(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(b))
    d1 = BatchNormalization()(d1)
    d1 = Dropout(0.5)(d1)
    d1 = Concatenate()([d1, e7])
    d2 = Conv2DTranspose(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(d1))
    d2 = BatchNormalization()(d2)
    d2 = Dropout(0.5)(d2)
    d2 = Concatenate()([d2, e6])
    d3 = Conv2DTranspose(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(d2))
    d3 = BatchNormalization()(d3)
    d3 = Concatenate()([d3, e5])
    d4 = Conv2DTranspose(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(d3))
    d4 = BatchNormalization()(d4)
    d4 = Concatenate()([d4, e4])
    d5 = Conv2DTranspose(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(d4))
    d5 = BatchNormalization()(d5)
    d5 = Concatenate()([d5, e3])
    d6 = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(d5))
    d6 = BatchNormalization()(d6)
    d6 = Concatenate()([d6, e2])
    d7 = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(d6))
    d7 = BatchNormalization()(d7)
    d7 = Concatenate()([d7, e1])
    
    # Output
    g = Conv2DTranspose(image_shape[2], (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('tanh')(d7))
    out_image = Activation('tanh')(g)
    
    model = Model(in_image, out_image)
    return model


In [11]:
def define_gan(g_model, d_model, image_shape):
    # Make weights in the discriminator not trainable
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False

    # Define the source image
    in_src = Input(shape=image_shape)
    
    # Supply the image as input to the generator 
    gen_out = g_model(in_src)
    
    # Supply the input image and generated image as inputs to the discriminator
    dis_out = d_model([in_src, gen_out])
    
    # Combine source image input, generated image, and discriminator output into a model
    model = Model(in_src, [dis_out, gen_out])
    
    return model



In [12]:


def define_combined_model(g_model, d_model, image_shape):
    # Define the source image
    in_src = Input(shape=image_shape)
    
    # Supply the image as input to the generator 
    gen_out = g_model(in_src)
    
    # Supply the input image and generated image as inputs to the discriminator
    dis_out = d_model([in_src, gen_out])
    
    # Combine source image input, generated image, and discriminator output into a model
    model = Model(in_src, [dis_out, gen_out])
    
    # Define GAN model
    gan_model = define_gan(g_model, d_model, image_shape)

    # Compile the GAN model
    opt = Adam(lr=0.0002, beta_1=0.5)
    gan_model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1, 100])

    return model


In [13]:
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape):
	# unpack dataset
	trainA, trainB = dataset
	# choose random instances
	ix = randint(0, trainA.shape[0], n_samples)
	# retrieve selected images
	X1, X2 = trainA[ix], trainB[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, patch_shape, patch_shape, 1))
	return [X1, X2], y

In [14]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y


In [15]:

# generate samples and save as a plot and save the model
#GAN models do not converge, we just want to find a good balance between
#the generator and the discriminator. Therefore, it makes sense to periodically
#save the generator model and check how good the generated image looks. 

import os
def summarize_performance(step, g_model, dataset, n_samples=3):
    print("Saving images...")
    # select a sample of input images
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
    # generate a batch of fake samples
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
    # scale all pixels from [-1,1] to [0,1]
    X_realA = (X_realA + 1) / 2.0
    X_realB = (X_realB + 1) / 2.0
    X_fakeB = (X_fakeB + 1) / 2.0
    # plot real source images
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + i)
        plt.axis('off')
        plt.imshow(X_realA[i])
    # plot generated target image
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + n_samples + i)
        plt.axis('off')
        plt.imshow(X_fakeB[i])
    # plot real target image
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + n_samples*2 + i)
        plt.axis('off')
        plt.imshow(X_realB[i])
    # save plot to file
    filename1 = os.path.join("C:\\Users\\net pc\\Desktop\\Thesis Paper 3\\generated_Pix2Pix", 'plot_%06d.png' % (step+1))
    plt.savefig(filename1)
    plt.close()
    # save the generator model
    filename2 = os.path.join("C:\\Users\\net pc\\Desktop\\Thesis Paper 3\\generated_Pix2Pix", 'model_%06d.h5' % (step+1))
    g_model.save(filename2)
    filename1 = os.path.join("C:\\Users\\net pc\\Desktop\\Thesis Paper 3\\generated_Pix2Pix", 'plot_%06d.png' % (step+1))
    filename2 = os.path.join("C:\\Users\\net pc\\Desktop\\Thesis Paper 3\\generated_Pix2Pix", 'model_%06d.h5' % (step+1))
    print("Image saved at:", filename1)
    print("Model saved at:", filename2)
    print('>Saved: %s and %s' % (filename1, filename2))

    


    


In [16]:

# train pix2pix models
def train(d_model, g_model, gan_model, dataset, n_epochs=100, n_batch=1):
	# determine the output square shape of the discriminator
	n_patch = d_model.output_shape[1]
	# unpack dataset
	trainA, trainB = dataset
	# calculate the number of batches per training epoch
	bat_per_epo = int(len(trainA) / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# manually enumerate epochs
	for i in range(n_steps):
		# select a batch of real samples
		[X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
		# generate a batch of fake samples
		X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
		# update discriminator for real samples
		d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
		# update discriminator for generated samples
		d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
		# update the generator
		g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
		# summarize performance
		print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
		# summarize model performance
		if (i+1) % (bat_per_epo * 10) == 0:
			summarize_performance(i, g_model, dataset)


In [17]:
# Define discriminator model
d_model = define_discriminator(target_shape)

# Define generator model
g_model = define_generator(target_shape)

# Define GAN model
gan_model = define_gan(g_model, d_model, target_shape)

# Define combined model
combined_model = define_combined_model(g_model, d_model, target_shape)



C:\Users\Public\Efendim\Lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [18]:
# Define GAN model
gan_model = define_gan(g_model, d_model, target_shape)

# Compile the GAN model
opt = Adam(lr=0.0002, beta_1=0.5)
gan_model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1, 100])

# Train Pix2Pix models using the GAN model
train(d_model, g_model, gan_model, data, n_epochs=100, n_batch=1)


1/1 [==============================] - 0s 309ms/step



>1, d1[0.379] d2[2.437] g[39.260]
1/1 [==============================] - 0s 75ms/step
>2, d1[1.142] d2[2.570] g[29.904]
1/1 [==============================] - 0s 82ms/step
>3, d1[0.688] d2[1.113] g[26.228]
1/1 [==============================] - 0s 71ms/step
>4, d1[0.279] d2[0.613] g[18.209]
1/1 [==============================] - 0s 72ms/step
>5, d1[0.427] d2[0.287] g[14.098]
1/1 [==============================] - 0s 89ms/step
>6, d1[0.623] d2[0.218] g[14.459]
1/1 [==============================] - 0s 80ms/step
>7, d1[0.629] d2[0.252] g[20.297]
1/1 [==============================] - 0s 75ms/step
>8, d1[0.484] d2[0.345] g[18.451]
1/1 [==============================] - 0s 72ms/step
>9, d1[0.372] d2[0.415] g[12.130]
1/1 [==============================] - 0s 73ms/step
>10, d1[0.311] d2[0.453] g[7.632]
1/1 [==============================] - 0s 74ms/step
>11, d1[0.291] d2[0.448] g[10.958]
1/1 [==============================] - 0s 73ms/step
>12, d1[0.316] d2[0.397] g[14.055]
1/1 [=============

1/1 [==============================] - 0s 68ms/step
>96, d1[0.017] d2[0.012] g[13.461]
1/1 [==============================] - 0s 84ms/step
>97, d1[0.003] d2[0.013] g[26.345]
1/1 [==============================] - 0s 89ms/step
>98, d1[0.014] d2[0.013] g[13.709]
1/1 [==============================] - 0s 74ms/step
>99, d1[0.005] d2[0.012] g[13.529]
1/1 [==============================] - 0s 74ms/step
>100, d1[0.004] d2[0.011] g[19.894]
1/1 [==============================] - 0s 74ms/step
>101, d1[0.012] d2[0.016] g[11.496]
1/1 [==============================] - 0s 73ms/step
>102, d1[0.004] d2[0.009] g[11.153]
1/1 [==============================] - 0s 75ms/step
>103, d1[0.004] d2[0.009] g[9.591]
1/1 [==============================] - 0s 77ms/step
>104, d1[0.008] d2[0.009] g[15.145]
1/1 [==============================] - 0s 70ms/step
>105, d1[0.008] d2[0.007] g[18.451]
1/1 [==============================] - 0s 94ms/step
>106, d1[0.005] d2[0.007] g[16.441]
1/1 [==============================] 

1/1 [==============================] - 0s 81ms/step
>189, d1[0.001] d2[0.002] g[19.562]
1/1 [==============================] - 0s 110ms/step
>190, d1[0.001] d2[0.002] g[18.352]
1/1 [==============================] - 0s 81ms/step
>191, d1[0.001] d2[0.002] g[22.561]
1/1 [==============================] - 0s 79ms/step
>192, d1[0.002] d2[0.002] g[9.227]
1/1 [==============================] - 0s 82ms/step
>193, d1[0.001] d2[0.001] g[26.195]
1/1 [==============================] - 0s 78ms/step
>194, d1[0.001] d2[0.003] g[14.284]
1/1 [==============================] - 0s 77ms/step
>195, d1[0.011] d2[0.002] g[18.593]
1/1 [==============================] - 0s 78ms/step
>196, d1[0.001] d2[0.006] g[14.679]
1/1 [==============================] - 0s 83ms/step
>197, d1[0.001] d2[0.013] g[12.039]
1/1 [==============================] - 0s 77ms/step
>198, d1[0.001] d2[0.002] g[14.815]
1/1 [==============================] - 0s 74ms/step
>199, d1[0.005] d2[0.003] g[3752.406]
1/1 [=========================

1/1 [==============================] - 0s 73ms/step
>282, d1[0.004] d2[0.004] g[13.472]
1/1 [==============================] - 0s 76ms/step
>283, d1[0.004] d2[0.004] g[16.163]
1/1 [==============================] - 0s 99ms/step
>284, d1[0.005] d2[0.008] g[11.671]
1/1 [==============================] - 0s 74ms/step
>285, d1[0.005] d2[0.003] g[3756.474]
1/1 [==============================] - 0s 78ms/step
>286, d1[0.004] d2[0.003] g[12.894]
1/1 [==============================] - 0s 69ms/step
>287, d1[0.002] d2[0.003] g[17.695]
1/1 [==============================] - 0s 72ms/step
>288, d1[0.005] d2[0.021] g[17.109]
1/1 [==============================] - 0s 89ms/step
>289, d1[0.004] d2[0.003] g[19.915]
1/1 [==============================] - 0s 80ms/step
>290, d1[0.009] d2[0.003] g[15.508]
1/1 [==============================] - 0s 86ms/step
>291, d1[0.002] d2[0.002] g[12.483]
1/1 [==============================] - 0s 83ms/step
>292, d1[0.002] d2[0.002] g[14.028]
1/1 [=========================

...

1/1 [==============================] - 0s 75ms/step
>9648, d1[0.290] d2[0.161] g[4.011]
1/1 [==============================] - 0s 72ms/step
>9649, d1[0.039] d2[0.363] g[5.093]
1/1 [==============================] - 0s 69ms/step
>9650, d1[0.154] d2[0.513] g[2.442]
1/1 [==============================] - 0s 92ms/step
>9651, d1[0.344] d2[0.201] g[4.405]
1/1 [==============================] - 0s 72ms/step
>9652, d1[0.070] d2[0.788] g[5.105]
1/1 [==============================] - 0s 73ms/step
>9653, d1[0.355] d2[0.116] g[3.451]
1/1 [==============================] - 0s 75ms/step
>9654, d1[0.153] d2[0.119] g[3.786]
1/1 [==============================] - 0s 73ms/step
>9655, d1[0.165] d2[0.115] g[4.463]
1/1 [==============================] - 0s 81ms/step
>9656, d1[0.425] d2[0.090] g[4.383]
1/1 [==============================] - 0s 69ms/step
>9657, d1[0.500] d2[0.172] g[3.431]
1/1 [==============================] - 0s 72ms/step
>9658, d1[0.158] d2[0.429] g[4.044]
1/1 [===========================

1/1 [==============================] - 0s 74ms/step
>9741, d1[0.252] d2[0.204] g[3.136]
1/1 [==============================] - 0s 72ms/step
>9742, d1[0.062] d2[0.239] g[3.502]
1/1 [==============================] - 0s 78ms/step
>9743, d1[0.083] d2[0.173] g[5.113]
1/1 [==============================] - 0s 69ms/step
>9744, d1[0.521] d2[0.167] g[4.542]
1/1 [==============================] - 0s 74ms/step
>9745, d1[0.168] d2[0.113] g[2.707]
1/1 [==============================] - 0s 69ms/step
>9746, d1[0.325] d2[0.134] g[4.532]
1/1 [==============================] - 0s 72ms/step
>9747, d1[0.048] d2[0.394] g[3.409]
1/1 [==============================] - 0s 74ms/step
>9748, d1[0.250] d2[0.094] g[4.318]
1/1 [==============================] - 0s 73ms/step
>9749, d1[0.154] d2[0.295] g[3.438]
1/1 [==============================] - 0s 96ms/step
>9750, d1[0.082] d2[0.264] g[2.658]
1/1 [==============================] - 0s 71ms/step
>9751, d1[0.100] d2[0.164] g[4.340]
1/1 [===========================

1/1 [==============================] - 0s 70ms/step
>9834, d1[0.889] d2[0.107] g[4.647]
1/1 [==============================] - 0s 72ms/step
>9835, d1[0.374] d2[0.120] g[5.368]
1/1 [==============================] - 0s 70ms/step
>9836, d1[0.654] d2[0.179] g[4.145]
1/1 [==============================] - 0s 74ms/step
>9837, d1[0.415] d2[0.238] g[3.705]
1/1 [==============================] - 0s 69ms/step
>9838, d1[0.412] d2[0.297] g[3.308]
1/1 [==============================] - 0s 69ms/step
>9839, d1[0.135] d2[0.502] g[3.949]
1/1 [==============================] - 0s 76ms/step
>9840, d1[0.064] d2[0.366] g[4.168]
1/1 [==============================] - 0s 72ms/step
>9841, d1[0.119] d2[0.465] g[5.390]
1/1 [==============================] - 0s 72ms/step
>9842, d1[0.176] d2[0.402] g[3.422]
1/1 [==============================] - 0s 73ms/step
>9843, d1[0.021] d2[0.185] g[3691.568]
1/1 [==============================] - 0s 72ms/step
>9844, d1[0.449] d2[0.161] g[4.299]
1/1 [========================

1/1 [==============================] - 0s 74ms/step
>9927, d1[0.214] d2[0.207] g[5.406]
1/1 [==============================] - 0s 73ms/step
>9928, d1[0.124] d2[0.258] g[4.490]
1/1 [==============================] - 0s 77ms/step
>9929, d1[0.039] d2[0.216] g[18.185]
1/1 [==============================] - 0s 69ms/step
>9930, d1[0.054] d2[0.131] g[6.764]
1/1 [==============================] - 0s 81ms/step
>9931, d1[0.116] d2[0.118] g[5.326]
1/1 [==============================] - 0s 70ms/step
>9932, d1[0.046] d2[0.103] g[4.174]
1/1 [==============================] - 0s 74ms/step
>9933, d1[0.692] d2[0.141] g[4.162]
1/1 [==============================] - 0s 74ms/step
>9934, d1[0.288] d2[0.102] g[4.058]
1/1 [==============================] - 0s 70ms/step
>9935, d1[0.548] d2[0.153] g[3.751]
1/1 [==============================] - 0s 72ms/step
>9936, d1[0.124] d2[0.693] g[3.089]
1/1 [==============================] - 0s 73ms/step
>9937, d1[0.220] d2[0.491] g[2.770]
1/1 [==========================

Image saved at: C:\Users\net pc\Desktop\Thesis Paper 3\generated_Pix2Pix\plot_010000.png
Model saved at: C:\Users\net pc\Desktop\Thesis Paper 3\generated_Pix2Pix\model_010000.h5
>Saved: C:\Users\net pc\Desktop\Thesis Paper 3\generated_Pix2Pix\plot_010000.png and C:\Users\net pc\Desktop\Thesis Paper 3\generated_Pix2Pix\model_010000.h5


In [19]:
import os

# Specify the directory where you want to test file writing
output_dir = "C:\\Users\\net pc\\Desktop\\Thesis Paper 3\\generated_Pix2Pix"

# Check if the directory exists, if not, create it
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Create a sample text file and write some content to it
file_path = os.path.join(output_dir, "test_file.txt")
with open(file_path, "w") as f:
    f.write("This is a test file.")

print("Test file has been written successfully at:", file_path)


Test file has been written successfully at: C:\Users\net pc\Desktop\Thesis Paper 3\generated_Pix2Pix\test_file.txt
